#  Fitting Demo
Example for fitting signal/background.
This example can be executed with:

```cpp
root > .x FittingDemo.C  (using the CINT interpreter)
root > .x FittingDemo.C+ (using the native complier via ACLIC)
```




**Author:** Rene Brun  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 01:30 AM.</small></i>

In [1]:
%%cpp -d
#include "TH1.h"
#include "TMath.h"
#include "TF1.h"
#include "TLegend.h"
#include "TCanvas.h"

  Quadratic background function
 

In [2]:
%%cpp -d
Double_t background(Double_t *x, Double_t *par) {
   return par[0] + par[1]*x[0] + par[2]*x[0]*x[0];
}

  Lorenzian Peak function
 

In [3]:
%%cpp -d
Double_t lorentzianPeak(Double_t *x, Double_t *par) {
  return (0.5*par[0]*par[1]/TMath::Pi()) /
    TMath::Max( 1.e-10,(x[0]-par[2])*(x[0]-par[2])
   + .25*par[1]*par[1]);
}

  Sum of background and peak function
 

In [4]:
%%cpp -d
Double_t fitFunction(Double_t *x, Double_t *par) {
  return background(x,par) + lorentzianPeak(x,&par[3]);
}

In [5]:
 //Bevington Exercise by Peter Malzacher, modified by Rene Brun

const int nBins = 60;

Double_t data[nBins] = { 6, 1,10,12, 6,13,23,22,15,21,
                        23,26,36,25,27,35,40,44,66,81,
                        75,57,48,45,46,41,35,36,53,32,
                        40,37,38,31,36,44,42,37,32,32,
                        43,44,35,33,33,39,29,41,32,44,
                        26,39,29,35,32,21,21,15,25,15};
TCanvas *c1 = new TCanvas("c1","Fitting Demo",10,10,700,500);
c1->SetFillColor(33);
c1->SetFrameFillColor(41);
c1->SetGrid();

TH1F *histo = new TH1F("histo",
   "Lorentzian Peak on Quadratic Background",60,0,3);
histo->SetMarkerStyle(21);
histo->SetMarkerSize(0.8);
histo->SetStats(0);

for(int i=0; i < nBins;  i++) histo->SetBinContent(i+1,data[i]);

Create a tf1 with the range from 0 to 3 and 6 parameters

In [6]:
TF1 *fitFcn = new TF1("fitFcn",fitFunction,0,3,6);
fitFcn->SetNpx(500);
fitFcn->SetLineWidth(4);
fitFcn->SetLineColor(kMagenta);

First try without starting values for the parameters
 This defaults to 1 for each param.
 this results in an ok fit for the polynomial function
 however the non-linear part (lorenzian) does not
 respond well.

In [7]:
fitFcn->SetParameters(1,1,1,1,1,1);
histo->Fit("fitFcn","0");

 FCN=58.9284 FROM MIGRAD    STATUS=CONVERGED     618 CALLS         619 TOTAL
                     EDM=1.54329e-09    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   1.2 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -8.64715e-01   8.87889e-01   3.02210e-05  -3.15277e-06
   2  p1           4.58434e+01   2.64076e+00   6.35729e-04   1.78463e-05
   3  p2          -1.33214e+01   9.77307e-01  -1.31737e-04   3.73302e-05
   4  p3           1.38074e+01   2.20785e+00  -1.29864e-03  -9.22424e-06
   5  p4           1.72308e-01   3.72077e-02  -5.22394e-06  -1.45631e-03
   6  p5           9.87281e-01   1.13098e-02   2.92804e-06  -3.44378e-04


Second try: set start values for some parameters

In [8]:
fitFcn->SetParameter(4,0.2); // width
fitFcn->SetParameter(5,1);   // peak

histo->Fit("fitFcn","V+","ep");

 **********
 **   10 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 p0          -8.64715e-01  8.87889e-01     no limits
     2 p1           4.58434e+01  2.64076e+00     no limits
     3 p2          -1.33214e+01  9.77307e-01     no limits
     4 p3           1.38074e+01  2.20785e+00     no limits
     5 p4           2.00000e-01  3.72077e-02     no limits
     6 p5           1.00000e+00  1.13098e-02     no limits
 **********
 **   11 **SET ERR           1
 **********
 **********
 **   12 **SET PRINT           2
 **********
 **********
 **   13 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **   14 **MIGRAD        1780        0.01
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-05
 FCN=60.858 FROM MIGRAD    STATUS=INITIATE       22 CALLS          23 TOTAL
     

Info in <TMinuitMinimizer::Minimize>: Finished to run MIGRAD - status 0


Improve the picture:

In [9]:
TF1 *backFcn = new TF1("backFcn",background,0,3,3);
backFcn->SetLineColor(kRed);
TF1 *signalFcn = new TF1("signalFcn",lorentzianPeak,0,3,3);
signalFcn->SetLineColor(kBlue);
signalFcn->SetNpx(500);
Double_t par[6];

Writes the fit results into the par array

In [10]:
fitFcn->GetParameters(par);

backFcn->SetParameters(par);
backFcn->Draw("same");

signalFcn->SetParameters(&par[3]);
signalFcn->Draw("same");

Draw the legend

In [11]:
TLegend *legend=new TLegend(0.6,0.65,0.88,0.85);
legend->SetTextFont(72);
legend->SetTextSize(0.04);
legend->AddEntry(histo,"Data","lpe");
legend->AddEntry(backFcn,"Background fit","l");
legend->AddEntry(signalFcn,"Signal fit","l");
legend->AddEntry(fitFcn,"Global Fit","l");
legend->Draw();

Draw all canvases 

In [12]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()